# Removing example unit tests from APPS prompts
Original code to extract example unit tests from CodeRL repo - Copyright (c) 2022, salesforce.com, inc.
Adapted to work with APPS dataset from huggingface/datasets, instead of having to manually download the APPS dataset.

# Reasoning
The test example cases in the train dataset seem to be the same as the hidden tests in a lot of cases, so we want to remove them to avoid overfitting to the hidden test examples.

In [1]:
# First party imports
import json, os, random, io, pdb
from tqdm import tqdm
from random import random
import numpy as np
import glob 
import pickle as pkl 
import re

# Second party imports
from apps_dataset import load_apps_dataset, APPSTask

C:\Users\noahv\.conda\envs\thesis\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the APPS dataset
apps_dataset = load_apps_dataset()

train_dataset = apps_dataset["train"]
test_dataset = apps_dataset["test"]

### IMPORTANT: SET TO DESIRED DATASET PORTION
target_dataset = train_dataset

No config specified, defaulting to: apps/all
Found cached dataset apps (C:/Users/noahv/.cache/huggingface/datasets/codeparrot___apps/all/0.0.0/04ac807715d07d6e5cc580f59cdc8213cd7dc4529d0bb819cca72c9f8e8c1aa5)
100%|██████████| 2/2 [00:00<00:00,  8.89it/s]


# Debugging

In [19]:
from data.apps.apps_dataset import APPSTask, construct_prompt_from_task

train_example_id = 2000

print(construct_prompt_from_task(train_dataset[train_example_id], remove_example_test=False))



case: APPSTask = train_dataset[train_example_id]

print(case["problem_id"])
case["input_output"]

Question:
Codefortia is a small island country located somewhere in the West Pacific. It consists of $n$ settlements connected by $m$ bidirectional gravel roads. Curiously enough, the beliefs of the inhabitants require the time needed to pass each road to be equal either to $a$ or $b$ seconds. It's guaranteed that one can go between any pair of settlements by following a sequence of roads.

Codefortia was recently struck by the financial crisis. Therefore, the king decided to abandon some of the roads so that:

  it will be possible to travel between each pair of cities using the remaining roads only,  the sum of times required to pass each remaining road will be minimum possible (in other words, remaining roads must form minimum spanning tree, using the time to pass the road as its weight),  among all the plans minimizing the sum of times above, the time required to travel between the king's residence (in settlement $1$) and the parliament house (in settlement $p$) using the remaining

'{\n  "inputs": [\n    "5 5 20 25\\n1 2 25\\n2 3 25\\n3 4 20\\n4 5 20\\n5 1 20\\n",\n    "6 7 13 22\\n1 2 13\\n2 3 13\\n1 4 22\\n3 4 13\\n4 5 13\\n5 6 13\\n6 1 13\\n",\n    "2 1 1 2\\n2 1 1\\n",\n    "2 1 9999999 10000000\\n1 2 10000000\\n",\n    "3 3 78422 6789101\\n3 1 6789101\\n2 1 78422\\n2 3 78422\\n",\n    "3 3 2770628 3912422\\n1 2 2770628\\n2 3 2770628\\n1 3 3912422\\n",\n    "3 3 2566490 5132980\\n1 2 2566490\\n2 3 2566490\\n3 1 5132980\\n",\n    "3 2 509529 5982470\\n1 2 509529\\n3 2 509529\\n",\n    "3 2 1349740 8457492\\n2 1 1349740\\n3 1 1349740\\n",\n    "3 2 150319 5002968\\n3 2 150319\\n1 2 5002968\\n",\n    "3 2 990530 8623767\\n3 2 8623767\\n1 2 990530\\n",\n    "3 2 810925 2022506\\n1 2 2022506\\n1 3 810925\\n",\n    "3 2 1651136 5131013\\n1 2 5131013\\n3 2 5131013\\n",\n    "3 2 451715 1577270\\n1 3 1577270\\n1 2 1577270\\n",\n    "3 3 1291926 4943478\\n2 3 1291926\\n1 2 1291926\\n3 1 1291926\\n",\n    "3 3 2132137 9084127\\n1 2 2132137\\n3 2 9084127\\n3 1 2132137\\